In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import current_thread
import os
import time

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

# Default configuration parameters - single source of truth
DEFAULT_CONFIG = {
    'backend': 'ns3-direct',
    'snark1_pull': True,
    'snark1_half_direct': True,
    'shuffle': False,
    'random_seed': 42,
    'group_count': 8,
    'group_validator_count': 1024,
    'group_local_aggregator_count': "10%",  # Default to 10% of group validators
    'global_aggregator_count': 102,
    'mesh_n': 8,
    'non_mesh_n': 4,
    'signature_time': '20ms',
    'signature_size': 3072,
    'snark_size': 131072,
    'snark1_threshold': 0.9,
    'snark2_threshold': 0.66,
    'aggregation_rate_per_sec': 1000,
    'snark_recursion_aggregation_rate_per_sec': 10,
    'pq_signature_verification_time': '30us',
    'snark_proof_verification_time': '5ms',
    'gml': 'shadow-atlas.bin',
    'max_bitrate': '50Mbps'  # Default EIP-7870 bandwidth limit
}


def generate_yaml_config(topology="gossip", **overrides):
    """
    Generate a YAML configuration with default values and optional overrides.
    
    Args:
        topology: Network topology ("gossip", "grid", etc.)
        **overrides: Dictionary of parameter overrides
        
    Common override parameters:
        - signature_size: Size of signatures in bytes
        - max_bitrate: Bandwidth limit (e.g., "10Mbps" or None for unlimited)
        - mesh_n: Gossip mesh_n parameter
        - non_mesh_n: Gossip non_mesh_n parameter
        - group_validator_count: Number of validators per group
        - group_local_aggregator_count: Number of local aggregators per group
        - group_count: Number of groups
        - global_aggregator_count: Number of global aggregators
        - snark1_threshold: Threshold for SNARK1 generation
        - aggregation_rate_per_sec: Rate of signature aggregation
        - random_seed: Random seed for reproducibility
    """
    
    # Use default configuration and add topology, then apply overrides
    defaults = DEFAULT_CONFIG.copy()
    defaults['topology'] = topology
    
    # Apply overrides
    config = {**defaults, **overrides}
    
    # Build the YAML string
    yaml_content = f"""
# Simulation Backend Configuration
backend: {config['backend']}
                                
snark1_pull: {str(config['snark1_pull']).lower()}
snark1_half_direct: {str(config['snark1_half_direct']).lower()}

# Network Topology Configuration
topology: {config['topology']}

# Whether to shuffle validators from the same group to different routers
shuffle: {str(config['shuffle']).lower()}

# Seed for reproducible simulation results
random_seed: {config['random_seed']}

# Role Assignment Configuration
roles:
  group_count: {config['group_count']}
  group_validator_count: {config['group_validator_count']}
  group_local_aggregator_count: {config['group_local_aggregator_count']}
  global_aggregator_count: {config['global_aggregator_count']}

# Gossipsub Network Configuration
gossip:
  mesh_n: {config['mesh_n']}
  non_mesh_n: {config['non_mesh_n']}

# Cryptographic Constants
consts:
  signature_time: {config['signature_time']}
  signature_size: {config['signature_size']}
  snark_size: {config['snark_size']}
  snark1_threshold: {config['snark1_threshold']}
  snark2_threshold: {config['snark2_threshold']}
  aggregation_rate_per_sec: {config['aggregation_rate_per_sec']}
  snark_recursion_aggregation_rate_per_sec: {config['snark_recursion_aggregation_rate_per_sec']}
  pq_signature_verification_time: {config['pq_signature_verification_time']}
  snark_proof_verification_time: {config['snark_proof_verification_time']}

# Network Simulation Parameters
network:
  gml: "{config['gml']}"
"""
    
    # Add bandwidth constraint if specified
    if config['max_bitrate'] is not None:
        yaml_content += f"  max_bitrate: {config['max_bitrate']}\n"
    
    return beamsim.yaml(yaml_content.strip())


# Worker function for parallel processing - using ThreadPoolExecutor instead of multiprocessing
# to avoid pickle issues in Jupyter notebooks
def _run_single_simulation_thread(topology, param_value, parameter_name, param_display, 
                                 config_overrides, run_kwargs, display_name, display_unit):
    """
    Worker function to run a single simulation in parallel using threading.
    
    Args:
        Individual parameters instead of tuple to avoid pickle issues
    
    Returns:
        Dictionary with simulation results
    """
    try:
        # Create config with this parameter value
        current_config = {**config_overrides, parameter_name: param_value}
        modified_yaml = generate_yaml_config(topology=topology, **current_config)
        
        # Run simulation with modified config
        current_run_kwargs = {**run_kwargs, 'c': modified_yaml}
        items = beamsim.run(**current_run_kwargs, t=topology, local_aggregation_only=True)
        
        # Extract SNARK1 timing information
        snark1_sent = beamsim.get_snark1_sent(items)
        if snark1_sent and len(snark1_sent[0]) > 0:
            snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
            
            # Get simulation info
            _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, "info")[0]
            
            return {
                'topology': topology,
                'topology_display': beamsim.topology_name[topology],
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'snark1_completion_time': snark1_completion_time,
                'validator_count': validator_count,
                'snark1_threshold': snark1_threshold,
                'success': True
            }
        else:
            return {
                'topology': topology,
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'success': False,
                'error': 'No SNARK1 data found'
            }
    
    except Exception as e:
        return {
            'topology': topology,
            'parameter_name': parameter_name,
            'parameter_value': param_value,
            'parameter_display': param_display,
            'success': False,
            'error': str(e)
        }

def theoretical_minimum_snark1_time(validator_count, snark1_threshold, signature_size, sig_agg_rate, max_bitrate):
    """
    Calculate the theoretical minimum SNARK1 aggregation time based on parameters.
    
    Args:
        validator_count: Total number of validators
        snark1_threshold: Threshold for SNARK1 generation (0-1)
        signature_size: Size of each signature in bytes
        sig_agg_rate: Rate of signature aggregation per second
        max_bitrate: Maximum bandwidth in bytes per second (None for unlimited)
        
    Returns:
        Theoretical minimum SNARK1 aggregation time in milliseconds
    """
    if max_bitrate is None:
        max_bitrate = float('inf')  # No limit
    
    # Calculate total data size to aggregate
    total_data_size = validator_count * signature_size * snark1_threshold  # in bytes

    # Calculate time to aggregate at the given rate
    agg_time_sec = validator_count * snark1_threshold  / sig_agg_rate  # in seconds

    # Calculate time based on bandwidth limit
    bandwidth_time_sec = total_data_size / max_bitrate  # in seconds
    
    min_time_sec = agg_time_sec + bandwidth_time_sec
    
    return min_time_sec * 1000  # Convert to milliseconds

def run_parameter_analysis(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                          display_name=None, display_unit="", parallel=True, max_workers=None, 
                          **base_config):
    """
    Generic function to analyze the effect of changing a parameter on SNARK1 aggregation time.
    
    Args:
        parameter_name: Name of the parameter to vary (e.g., 'signature_size', 'mesh_n')
        parameter_values: List of values to test for the parameter
        topologies: List of network topologies to test
        display_name: Human-readable name for the parameter (defaults to parameter_name)
        display_unit: Unit to display after the parameter value (e.g., "KB", "Mbps")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: min(8, cpu_count()))
        **base_config: Base configuration parameters and run kwargs
        
    Returns:
        DataFrame with results, with parameter order preserved
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Separate run kwargs from config overrides
    run_kwargs_keys = {'mpi', 'c', 'b', 'g', 'gv', 'la', 'ga', 'shuffle', 't', 'local_aggregation_only'}
    run_kwargs = {k: v for k, v in base_config.items() if k in run_kwargs_keys}
    config_overrides = {k: v for k, v in base_config.items() if k not in run_kwargs_keys}
    
    # Determine the number of workers (use ThreadPoolExecutor which works better in Jupyter)
    if max_workers is None:
        max_workers = min(8, len(parameter_values) * len(topologies))  # Conservative default
    
    print(f"Testing {display_name}: {parameter_values}")
    print(f"Topologies: {topologies}")
    
    if parallel:
        print(f"Running simulations in parallel with {max_workers} workers (ThreadPoolExecutor)...\n")
    else:
        print("Running simulations sequentially...\n")
    
    # Prepare all simulation parameters
    simulation_params = []
    for topology in topologies:
        for param_value in parameter_values:
            param_display = f"{param_value} {display_unit}".strip() if param_value is not None else "Default/Unlimited"
            simulation_params.append((
                topology, param_value, parameter_name, param_display,
                config_overrides, run_kwargs, display_name, display_unit
            ))
    
    results = []
    
    if parallel and len(simulation_params) > 1:
        # Run simulations in parallel using ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_params = {}
            for params in simulation_params:
                topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
                future = executor.submit(_run_single_simulation_thread, 
                                       topology, param_value, parameter_name, param_display,
                                       config_overrides, run_kwargs, display_name, display_unit)
                future_to_params[future] = params
            
            # Process completed tasks
            total_sims = len(simulation_params)
            completed = 0
            
            for future in as_completed(future_to_params):
                completed += 1
                result = future.result()
                
                if result['success']:
                    results.append(result)
                    topology_display = beamsim.topology_name.get(result['topology'], result['topology'])
                    print(f"  [{completed}/{total_sims}] {topology_display} - {result['parameter_display']}: "
                          f"{result['snark1_completion_time']:.1f} ms")
                else:
                    print(f"  [{completed}/{total_sims}] ERROR - {result['topology']} - {result['parameter_display']}: "
                          f"{result['error']}")
    else:
        # Run simulations sequentially (original behavior)
        for i, params in enumerate(simulation_params):
            topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
            print(f"  [{i+1}/{len(simulation_params)}] Testing {topology} - {param_display}")
            
            result = _run_single_simulation_thread(
                topology, param_value, parameter_name, param_display,
                config_overrides, run_kwargs, display_name, display_unit
            )
            
            if result['success']:
                results.append(result)
                print(f"    → {result['snark1_completion_time']:.1f} ms")                
            else:
                    print(f"    → ERROR: {result['error']}")
    
    # Convert to DataFrame and preserve parameter order
    df = pd.DataFrame(results)
    if not df.empty:
        # Add an order column to preserve the original parameter order
        param_to_order = {param: i for i, param in enumerate(parameter_values)}
        df['parameter_order'] = df['parameter_value'].map(param_to_order)
        # Sort by topology and parameter order to ensure consistent ordering
        df = df.sort_values(['topology', 'parameter_order']).reset_index(drop=True)
    
    return df


def plot_parameter_analysis(results_df, parameter_name, display_name=None, display_unit="", 
                          title_suffix="", figure_size=(14, 8), include_theoretical=True,
                          **base_config):
    """
    Plot the results of a parameter analysis.
    
    Args:
        results_df: DataFrame from run_parameter_analysis
        parameter_name: Name of the parameter that was varied
        display_name: Human-readable name for the parameter
        display_unit: Unit for the parameter
        title_suffix: Additional text for the plot title
        figure_size: Size of the plot figure
        include_theoretical: Whether to include theoretical minimum line
        **base_config: Base configuration parameters for theoretical calculations
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Create the plot
    plt.figure(figsize=figure_size)
    
    # Plot lines for each topology
    topologies = results_df['topology'].unique()
    colors = sns.color_palette("husl", len(topologies))
    
    # Get parameter order from the dataframe (preserves original input order)
    if 'parameter_order' in results_df.columns:
        # Use the preserved order
        ordered_params = results_df.sort_values('parameter_order')['parameter_display'].unique()
    else:
        # Fallback to original behavior
        ordered_params = results_df['parameter_display'].unique()
    
    for i, topology in enumerate(topologies):
        topo_data = results_df[results_df['topology'] == topology].copy()
        
        # Sort by parameter order to maintain input sequence
        if 'parameter_order' in topo_data.columns:
            topo_data = topo_data.sort_values('parameter_order')
        else:
            # Fallback: try to sort by parameter value if numeric
            try:
                topo_data = topo_data.sort_values('parameter_value')
            except:
                pass  # Keep original order if sorting fails
        
        x_values = list(range(len(topo_data)))
        y_values = topo_data['snark1_completion_time'].tolist()
        
        plt.plot(
            x_values,
            y_values,
            marker='o',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=f'{topo_data.iloc[0]["topology_display"]} Topology',
            alpha=0.8
        )
    
    # Add theoretical minimum line if requested
    if include_theoretical and len(results_df) > 0:
        # Get default parameters for theoretical calculation
        # Use the global DEFAULT_CONFIG as the base for theoretical calculations
        theoretical_params = {**DEFAULT_CONFIG, **base_config}
        
        # Convert max_bitrate to bytes per second if it's a string
        max_bitrate = theoretical_params['max_bitrate']
        if isinstance(max_bitrate, str) and max_bitrate.endswith('Mbps'):
            max_bitrate_mbps = float(max_bitrate[:-4])
            max_bitrate_bps = max_bitrate_mbps * 1024 * 1024 / 8  # Convert Mbps to bytes/sec
        elif max_bitrate is None:
            max_bitrate_bps = None
        else:
            max_bitrate_bps = max_bitrate
        
        # Calculate total validator count (group_count * group_validator_count)
        total_validator_count = theoretical_params['group_validator_count']
        
        # Calculate theoretical minimum times for each parameter value
        theoretical_times = []
        sample_data = results_df.sort_values('parameter_order' if 'parameter_order' in results_df.columns else 'parameter_value')
        unique_param_values = sample_data['parameter_value'].unique()
        
        for param_value in unique_param_values:
            # Update the parameter being analyzed
            current_params = theoretical_params.copy()
            current_params[parameter_name] = param_value
            
            # Handle special cases for parameter-specific calculations
            if parameter_name == 'max_bitrate':
                if isinstance(param_value, str) and param_value.endswith('Mbps'):
                    max_bitrate_mbps = float(param_value[:-4])
                    current_max_bitrate = max_bitrate_mbps * 1024 * 1024 / 8
                elif param_value is None:
                    current_max_bitrate = None
                else:
                    current_max_bitrate = param_value
            else:
                current_max_bitrate = max_bitrate_bps
            
            # Recalculate total validator count if group parameters changed
            if parameter_name in ['group_count', 'group_validator_count']:
                current_total_validators = current_params['group_validator_count']
            else:
                current_total_validators = total_validator_count
            
            theoretical_time = theoretical_minimum_snark1_time(
                validator_count=current_total_validators,
                snark1_threshold=current_params['snark1_threshold'],
                signature_size=current_params['signature_size'],
                sig_agg_rate=current_params['aggregation_rate_per_sec'],
                max_bitrate=current_max_bitrate
            )
            theoretical_times.append(theoretical_time)
        
        # Plot theoretical minimum line
        x_values = list(range(len(theoretical_times)))
        plt.plot(
            x_values,
            theoretical_times,
            linestyle='--',
            linewidth=2,
            color='red',
            label='Theoretical Minimum',
            alpha=0.8
        )
    
    # Set axis labels and title
    plt.xlabel(f'{display_name} {display_unit}'.strip(), fontweight='bold', fontsize=12)
    plt.ylabel('SNARK1 Aggregation Time (ms)', fontweight='bold', fontsize=12)
    
    title = f'Effect of {display_name} on SNARK1 Aggregation Time'
    if title_suffix:
        title += f'\n{title_suffix}'
    title += '\n(Local Aggregation Only Mode)'
    plt.title(title, fontweight='bold', fontsize=14)
    
    # Set x-axis tick labels using preserved parameter order
    plt.xticks(range(len(ordered_params)), ordered_params, rotation=45)
    
    plt.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Add performance annotations
    if len(results_df) > 0:
        min_time = results_df['snark1_completion_time'].min()
        max_time = results_df['snark1_completion_time'].max()
        time_diff = max_time - min_time
        improvement = (time_diff / max_time) * 100
        
        plt.text(0.02, 0.98, 
                f'Max time difference: {time_diff:.1f}ms\n'
                f'Performance improvement: {improvement:.1f}%',
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print(f"{display_name.upper()} EFFECT SUMMARY")
    print("="*60)
    
    # Print theoretical minimum information if included
    if include_theoretical and len(results_df) > 0:
        print(f"\nTheoretical Minimum:")
        print(f"  SNARK1 time range: {min(theoretical_times):.1f} - {max(theoretical_times):.1f} ms")
        print(f"  Time difference: {max(theoretical_times) - min(theoretical_times):.1f} ms")
        
        # Show theoretical minimum for each parameter value
        sample_data = results_df.sort_values('parameter_order' if 'parameter_order' in results_df.columns else 'parameter_value')
        unique_param_displays = sample_data['parameter_display'].unique()
        for i, param_display in enumerate(unique_param_displays):
            print(f"    {param_display:>12}: {theoretical_times[i]:.1f} ms")
    
    for topology in topologies:
        topo_data = results_df[results_df['topology'] == topology]
        if len(topo_data) > 0:
            print(f"\n{topo_data.iloc[0]['topology_display']} Topology:")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  Time difference: {topo_data['snark1_completion_time'].max() - topo_data['snark1_completion_time'].min():.1f} ms")
            
            # Show performance for each parameter value
            for _, row in topo_data.iterrows():
                print(f"    {row['parameter_display']:>12}: {row['snark1_completion_time']:.1f} ms")


def analyze_parameter_effect(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                           display_name=None, display_unit="", title_suffix="", 
                           parallel=True, max_workers=None, **base_config):
    """
    Complete analysis workflow: run simulations and plot results.
    
    Args:
        parameter_name: Name of the parameter to vary
        parameter_values: List of values to test
        topologies: List of topologies to test
        display_name: Human-readable parameter name
        display_unit: Unit for the parameter
        title_suffix: Additional text for plot title
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration and run parameters
        
    Returns:
        DataFrame with results
    """
    
    # Run the analysis
    results_df = run_parameter_analysis(
        parameter_name, parameter_values, topologies, 
        display_name, display_unit, parallel=parallel, 
        max_workers=max_workers, **base_config
    )
    
    # Plot the results
    plot_parameter_analysis(
        results_df, parameter_name, display_name, 
        display_unit, title_suffix, **base_config
    )
    
    return results_df

In [ ]:
def analyze_signature_size_effect(signature_sizes, topologies=["gossip", "grid"], 
                                 parallel=True, max_workers=None, **base_config):
    """
    Analyze how signature size affects SNARK1 aggregation time using the refactored framework.
    
    Args:
        signature_sizes: List of signature sizes to test (in bytes)
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='signature_size',
        parameter_values=signature_sizes,
        topologies=topologies,
        display_name='Signature Size',
        display_unit='bytes',
        title_suffix='',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )

In [ ]:
def analyze_bandwidth_effect(bandwidth_limits, signature_size=3072, topologies=["gossip", "grid"], 
                            parallel=True, max_workers=None, **base_config):
    """
    Analyze how max incoming bandwidth affects SNARK1 aggregation time using the refactored framework.
    
    Args:
        bandwidth_limits: List of bandwidth limits to test (in Mbps, None for unlimited)
        signature_size: Fixed signature size to use (in bytes)
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    # Convert bandwidth values to the format expected by the YAML config
    bandwidth_values = []
    for bw in bandwidth_limits:
        if bw is None:
            bandwidth_values.append(None)
        else:
            bandwidth_values.append(f"{bw}Mbps")
    
    return analyze_parameter_effect(
        parameter_name='max_bitrate',
        parameter_values=bandwidth_values,
        topologies=topologies,
        display_name='Max Incoming Bandwidth',
        display_unit='',
        title_suffix=f'Signature Size: {signature_size/1024:.1f} KB',
        parallel=parallel,
        max_workers=max_workers,
        signature_size=signature_size,  # Fixed signature size
        **base_config
    )

In [ ]:
def analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], parallel=True, max_workers=None, **base_config):
    """
    Analyze how mesh_n (gossip mesh size) affects SNARK1 aggregation time.
    
    Args:
        mesh_n_values: List of mesh_n values to test
        topologies: List of network topologies to test (typically just "gossip")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='mesh_n',
        parameter_values=mesh_n_values,
        topologies=topologies,
        display_name='Mesh N',
        display_unit='peers',
        title_suffix='Gossip Network Configuration',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )


def analyze_non_mesh_n_effect(non_mesh_n_values, topologies=["gossip"], parallel=True, max_workers=None, **base_config):
    """
    Analyze how non_mesh_n (gossip non-mesh connections) affects SNARK1 aggregation time.
    
    Args:
        non_mesh_n_values: List of non_mesh_n values to test
        topologies: List of network topologies to test (typically just "gossip")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='non_mesh_n',
        parameter_values=non_mesh_n_values,
        topologies=topologies,
        display_name='Non-Mesh N',
        display_unit='peers',
        title_suffix='Gossip Network Configuration',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )


def analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], 
                                       parallel=True, max_workers=None, **base_config):
    """
    Analyze how group_validator_count affects SNARK1 aggregation time.
    
    Args:
        validator_counts: List of validator counts per group to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_validator_count',
        parameter_values=validator_counts,
        topologies=topologies,
        display_name='Group Validator Count',
        display_unit='validators',
        title_suffix='Network Scale Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )


def analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], 
                                        parallel=True, max_workers=None, **base_config):
    """
    Analyze how group_local_aggregator_count affects SNARK1 aggregation time.
    
    Args:
        aggregator_counts: List of local aggregator counts per group to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_local_aggregator_count',
        parameter_values=aggregator_counts,
        topologies=topologies,
        display_name='Local Aggregator Count',
        display_unit='aggregators',
        title_suffix='Aggregation Capacity Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )


def analyze_aggregation_rate_effect(aggregation_rates, topologies=["gossip", "grid"], 
                                  parallel=True, max_workers=None, **base_config):
    """
    Analyze how aggregation_rate_per_sec affects SNARK1 aggregation time.
    
    Args:
        aggregation_rates: List of aggregation rates to test (signatures per second)
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='aggregation_rate_per_sec',
        parameter_values=aggregation_rates,
        topologies=topologies,
        display_name='Aggregation Rate',
        display_unit='sig/sec',
        title_suffix='Processing Speed Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )

In [ ]:
topologies = ["gossip", "grid"]

# Signature Size Analysis - Using the refactored approach
# This demonstrates how the new framework simplifies the analysis process

# Test different signature sizes to analyze their effect on SNARK1 aggregation time
signature_sizes = [
    96,      # 96 bytes - very small signature (BLS)
    1024,    # 1 KB - small signature
    2048,    # 2 KB - medium-small signature  
    3072,    # 3 KB - default signature size
    4096,    # 4 KB - medium signature
]

# Base configuration parameters
base_config = {
    'mpi': False,  # Use MPI for faster simulations
    # Add any other base parameters you want to override
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of signature size on SNARK1 aggregation time...")
print("Using the refactored analysis framework for cleaner code.\n")

# Run the signature size effect analysis using the new framework
results_df = analyze_signature_size_effect(signature_sizes, topologies, max_workers=5, **base_config)

In [ ]:
# Analyze the effect of max incoming bandwidth on SNARK1 aggregation time
# Test different bandwidth limits to understand network bottlenecks

bandwidth_limits = [
    5,      # 5 Mbps - Low bandwidth
    10,     # 10 Mbps - Medium-low bandwidth
    25,     # 25 Mbps - Medium bandwidth
    50,     # 50 Mbps - High bandwidth
    100,    # 100 Mbps - Very high bandwidth
    200,    # 200 Mbps - Extremely high bandwidth
    None,   # No limit - Unlimited bandwidth
]

# Use fixed signature size of 3072 bytes (3KB) as requested
signature_size = 3072

# Base configuration parameters
base_config = {
    'mpi': False,  # Use MPI for faster simulations
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of max incoming bandwidth on SNARK1 aggregation time...")
print(f"Using fixed signature size: {signature_size} bytes ({signature_size/1024:.1f} KB)")
print("Using the refactored analysis framework.\n")

# Run the bandwidth effect analysis using the new framework
bandwidth_results_df = analyze_bandwidth_effect(bandwidth_limits, signature_size, topologies, max_workers=5, **base_config)

In [ ]:
# Example analyses for new parameters using the refactored framework
# These demonstrate how easy it is to analyze different parameters

# Example 1: Analyze mesh_n effect (gossip network mesh size)
print("="*60)
print("EXAMPLE 1: Mesh N Analysis")
print("="*60)

mesh_n_values = [4, 6, 8, 10, 12]  # Different mesh sizes
base_config = {'mpi': False, 'signature_size': 3072}

mesh_n_results = analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], max_workers=5, **base_config)


# Example 2: Analyze non_mesh_n effect (gossip non-mesh connections)
# print("\n" + "="*60)
# print("EXAMPLE 2: Non-Mesh N Analysis")
# print("="*60)

# non_mesh_n_values = [2, 4, 6, 8, 10]  # Different non-mesh connection counts
# base_config = {'mpi': False, 'signature_size': 3072}

# non_mesh_n_results = analyze_non_mesh_n_effect(non_mesh_n_values, topologies=["gossip"], max_workers=5, **base_config)


# Example 3: Analyze group_validator_count effect (network scale)
print("\n" + "="*60)
print("EXAMPLE 3: Group Validator Count Analysis")
print("="*60)

validator_counts = [128, 256, 512, 768, 1024]  # Different validator counts per group
base_config = {'mpi': False, 'signature_size': 3072}

validator_count_results = analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], **base_config)


# Example 4: Analyze local aggregator count effect
print("\n" + "="*60)
print("EXAMPLE 4: Local Aggregator Count Analysis")
print("="*60)

aggregator_counts = ["1%", "10%", "25%", "50%"]  # Different local aggregator counts
base_config = {'mpi': False, 'signature_size': 3072}

aggregator_count_results = analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], max_workers=5, **base_config)